# The Price is Right

Here I build a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

### I implement a some classes in deals.py that do the following:
1. **ScrapedDeal** scrapes the RSS feed to find deals and retains product description, url and the deal price
2. Defines **Deal** and **DealSelection** structured data in which I want openAI API to return the deal data so that I can process it later

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:32<00:00, 18.53s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Juggernaut Storage 16-Quart Heavy-Duty Milk Crate for $7 + pickup\nDetails: That's the lowest price we could find by $15. Opt for store pickup where available to avoid the $7 shipping fee, otherwise orders over $35 get free shipping. Buy Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Juggernaut-Storage-16-Quart-Heavy-Duty-Milk-Crate-for-7-pickup/21774855.html?iref=rss-c196"

### Prompts to get top 5 best described reliable deals!

#### Note:

Defining the output format in system and user prompts is not necessary because we can use the OpenAI API to get structured outpout (as I show later in the doc). However, to be extra strict with the model it is good to describe it in both places.

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Belkin BoostCharge 10,000mAh USB-C Portable Charger for $13 + free shipping w/ $35
Details: That's $17 less than Amazon charges, and the best price we've ever seen. Orders over $35 ship free, otherwise it adds $7. Buy Now at Walmart
Features: 1 USB-C port and 2 USB-A ports charge 3 devices simultaneously Model: BPB011
URL: https://www.dealnews.com/products/Belkin/Be

In [25]:
def get_recommendations():

    ## Earlier I was using the beta version but switched to production
    # completion = openai.beta.chat.completions.parse(
    #     model="gpt-4o-mini",
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": user_prompt}
    #   ],
    #     response_format=DealSelection #this is how get structured output so a format description is not necessary in the system and user prompt but its always good to describe it in both places.
    # )
    # result = completion.choices[0].message.parsed
    # return result

    
    ## The beta version is now in production, so using productionized call
    response = openai.responses.parse(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        text_format=DealSelection #this is how get structured output so a format description is not necessary in the system and user prompt but its always good to describe it in both places.
    )

    result = response.output_parsed
    return result

In [26]:
result = get_recommendations()

In [27]:
len(result.deals)

5

In [35]:
type(result)

agents.deals.DealSelection

In [36]:
result

DealSelection(deals=[Deal(product_description='The Belkin BoostCharge 10,000mAh USB-C Portable Charger is designed to keep your devices powered on the go. It features one USB-C port and two USB-A ports, allowing you to charge up to three devices simultaneously. With a compact and lightweight design, this charger is perfect for travel or everyday use. Engineered for durability and performance, it ensures your devices remain charged throughout the day.', price=13.0, url='https://www.dealnews.com/products/Belkin/Belkin-Boost-Charge-10-000-m-Ah-USB-C-Portable-Charger/494774.html?iref=rss-c142'), Deal(product_description='The Samsung UN58U7900FFXZA U7900 Series 58" 4K UHD LED Smart Tizen TV offers stunning 4K resolution for a cinematic viewing experience. This smart TV features Amazon Alexa compatibility, allowing you to control it with your voice. Equipped with Tizen OS, it provides access to a variety of streaming apps for endless entertainment. The sleek design and high-quality display m

In [16]:
type(result.deals)

list

In [28]:
result.deals[1]

Deal(product_description='The Samsung UN58U7900FFXZA U7900 Series 58" 4K UHD LED Smart Tizen TV offers stunning 4K resolution for a cinematic viewing experience. This smart TV features Amazon Alexa compatibility, allowing you to control it with your voice. Equipped with Tizen OS, it provides access to a variety of streaming apps for endless entertainment. The sleek design and high-quality display make it a perfect addition to any living space.', price=298.0, url='https://www.dealnews.com/products/Samsung/Samsung-UN58-U7900-FFXZA-U7900-Series-58-4-K-UHD-LED-Smart-Tizen-TV-2025/494771.html?iref=rss-c142')

In [30]:
for i in range(0,5):
    print(result.deals[i])
    print('\n-------\n\n')

product_description='The Belkin BoostCharge 10,000mAh USB-C Portable Charger is designed to keep your devices powered on the go. It features one USB-C port and two USB-A ports, allowing you to charge up to three devices simultaneously. With a compact and lightweight design, this charger is perfect for travel or everyday use. Engineered for durability and performance, it ensures your devices remain charged throughout the day.' price=13.0 url='https://www.dealnews.com/products/Belkin/Belkin-Boost-Charge-10-000-m-Ah-USB-C-Portable-Charger/494774.html?iref=rss-c142'

-------


product_description='The Samsung UN58U7900FFXZA U7900 Series 58" 4K UHD LED Smart Tizen TV offers stunning 4K resolution for a cinematic viewing experience. This smart TV features Amazon Alexa compatibility, allowing you to control it with your voice. Equipped with Tizen OS, it provides access to a variety of streaming apps for endless entertainment. The sleek design and high-quality display make it a perfect additio

# Now I agentify this!

In [3]:
from agents.scanner_agent import ScannerAgent

In [4]:
agent = ScannerAgent()
result = agent.scan()

In [14]:
type(result)

agents.deals.DealSelection

In [5]:
result

DealSelection(deals=[Deal(product_description='The Belkin BoostCharge 10,000mAh USB-C Portable Charger is a sleek and powerful device perfect for keeping your devices charged on the go. It features one USB-C port and two USB-A ports, allowing you to charge up to three devices simultaneously. Its compact design makes it easy to carry in your bag or pocket, ensuring you have power whenever you need it. This charger is ideal for travelers and those who use multiple devices throughout the day.', price=13.0, url='https://www.dealnews.com/products/Belkin/Belkin-Boost-Charge-10-000-m-Ah-USB-C-Portable-Charger/494774.html?iref=rss-c142'), Deal(product_description='The Samsung UN58U7900FFXZA U7900 Series 58" 4K UHD LED Smart Tizen TV offers stunning picture quality with its 4K resolution capabilities. It is equipped with smart technology that allows you to access your favorite streaming services like Netflix and Hulu easily. Additionally, it\'s Amazon Alexa compatible for voice control convenie

In [15]:
type(result.deals)

list

In [13]:
len(result.deals)

5

In [8]:
result.deals[1]

Deal(product_description='The Samsung UN58U7900FFXZA U7900 Series 58" 4K UHD LED Smart Tizen TV offers stunning picture quality with its 4K resolution capabilities. It is equipped with smart technology that allows you to access your favorite streaming services like Netflix and Hulu easily. Additionally, it\'s Amazon Alexa compatible for voice control convenience. This model brings an immersive viewing experience right into your living room, making it perfect for movie nights.', price=298.0, url='https://www.dealnews.com/products/Samsung/Samsung-UN58-U7900-FFXZA-U7900-Series-58-4-K-UHD-LED-Smart-Tizen-TV-2025/494771.html?iref=rss-c142')

In [9]:
result.deals[0]

Deal(product_description='The Belkin BoostCharge 10,000mAh USB-C Portable Charger is a sleek and powerful device perfect for keeping your devices charged on the go. It features one USB-C port and two USB-A ports, allowing you to charge up to three devices simultaneously. Its compact design makes it easy to carry in your bag or pocket, ensuring you have power whenever you need it. This charger is ideal for travelers and those who use multiple devices throughout the day.', price=13.0, url='https://www.dealnews.com/products/Belkin/Belkin-Boost-Charge-10-000-m-Ah-USB-C-Portable-Charger/494774.html?iref=rss-c142')

# So now I can get the Top 5 deals!